In [29]:
sentences = [
    "Bo is a dump ass.",
    "Do you love fishing?",
    "Dinosaurs are exstinct thousands of years ago.",
    "Iowa is the best state.",
    "The state beside Utah is of the first rank.",
    "The giant retile died out a long time ago",
]

model_name = 'all-MiniLM-L6-v2'

In [ ]:
!pip install -U transformers

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch

In [32]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [33]:
tokens = {'input_ids': [], 'attention_mask': []}

In [34]:
for sentence in sentences:
    new_tokens = tokenizer.encode_plus(sentence, max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt'
                                       )
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

In [35]:
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [36]:
tokens['input_ids'].shape

torch.Size([6, 128])

In [37]:
outputs = model(**tokens)

In [38]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [39]:
embeddings = outputs.last_hidden_state
embeddings.shape

torch.Size([6, 128, 384])

In [42]:
attention = tokens['attention_mask']
attention.shape

torch.Size([6, 128])

In [44]:
mask = attention.unsqueeze(-1).expand(embeddings.shape).float()

In [47]:
mask_embeddings = embeddings * mask

In [48]:
mask_embeddings.shape

torch.Size([6, 128, 384])

In [49]:
summed = torch.sum(mask_embeddings, 1)
summed.shape

torch.Size([6, 384])

In [52]:
counts = torch.clamp(mask.sum(1), min=1e-9)
counts.shape

torch.Size([6, 384])

In [53]:
mean_pooled = summed / counts
mean_pooled.shape

torch.Size([6, 384])

In [54]:
mean_pooled

tensor([[ 0.0007, -0.4034, -0.1385,  ..., -0.0523,  0.3028, -0.3839],
        [-0.2656,  0.2521,  0.0457,  ...,  0.0800, -0.2341, -0.1258],
        [-0.1167,  0.2341,  0.1494,  ..., -0.0304,  0.5418, -0.1775],
        [ 0.4493, -0.0272,  0.2857,  ...,  0.1059,  0.5939,  0.1048],
        [ 0.0388, -0.4498,  0.3170,  ..., -0.0562,  0.2609,  0.1691],
        [ 0.0281,  0.5835,  0.1200,  ..., -0.1988, -0.0484,  0.3738]],
       grad_fn=<DivBackward0>)

In [59]:
from sklearn.metrics.pairwise import cosine_similarity

In [62]:
cosine_similarity(
    [mean_pooled[2]],
    mean_pooled[3:]
)

array([[0.02624043, 0.0699896 , 0.41992646]], dtype=float32)